In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Dropout
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import rasterio as rio
from rasterio.plot import reshape_as_image
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from sklearn.model_selection import train_test_split
import glob,os,sys,cv2
from datetime import datetime
import visualkeras
from PIL import ImageFont

In [2]:
def double_conv_block(x, n_filters):
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    return x

def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)
    return f, p

def upsample_block(x, conv_features, n_filters):
    # upsample
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    # concatenate
    x = layers.concatenate([x, conv_features])
    # dropout
    x = layers.Dropout(0.3)(x)
    # Conv2D twice with ReLU activation
    x = double_conv_block(x, n_filters)
    return x

In [3]:
def build_unet_model():
    # inputs
    inputs = layers.Input(shape=(256,256,12))

    # encoder: contracting path - downsample
    # 1 - downsample
    f1, p1 = downsample_block(inputs, 64)
    # 2 - downsample
    f2, p2 = downsample_block(p1, 128)
    # 3 - downsample
    f3, p3 = downsample_block(p2, 256)
    # 4 - downsample
    f4, p4 = downsample_block(p3, 512)

    # 5 - bottleneck
    bottleneck = double_conv_block(p4, 1024)
    # model = keras.Sequential()
    # bn = tf.keras.Model(inputs,bottleneck)
    
    
    

    # decoder: expanding path - upsample
    # 6 - upsample
    u6 = upsample_block(bottleneck, f4, 512)
    # 7 - upsample
    u7 = upsample_block(u6, f3, 256)
    # 8 - upsample
    u8 = upsample_block(u7, f2, 128)
    # 9 - upsample
    u9 = upsample_block(u8, f1, 64)

    # outputs
    
    flat = layers.Flatten()(u9)
    reg_layer = layers.Dense(64, activation='relu')(flat) # Add another dense layer
    reg_layer = Dropout(0.5)(reg_layer)
    reg_layer = layers.Dense(32, activation='relu')(reg_layer)
    reg_layer = layers.Dense(16, activation='relu')(reg_layer)
    reg_layer = layers.Dense(8, activation='relu')(reg_layer)
    reg_layer = layers.Dense(4, activation='relu')(reg_layer)
    reg_layer = layers.Dense(1,activation='linear',name="reg_output")(reg_layer)
    
    
    # outputs = layers.Conv2D(1, 1, padding="same", activation = "softmax")(u9)
    outputs = layers.Conv2D(1, 1, padding="same", name="segmentation",activation = "sigmoid")(u9)

    # unet model with Keras Functional API
    unet_model = tf.keras.Model(inputs=inputs, outputs=[outputs,reg_layer], name="U-Net Multi Head")

    return unet_model


unet_model = build_unet_model()

2023-11-08 10:59:43.855558: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


In [4]:
unet_model.summary()

Model: "U-Net Multi Head"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                2)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  6976        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                  

In [5]:
from keras.utils.vis_utils import plot_model
plot_model(unet_model, to_file='Unet_multi_head_model_v2.png', show_shapes=True, show_layer_names=True)
mse = tf.keras.metrics.MeanSquaredError()
rmse = tf.keras.metrics.RootMeanSquaredError()
mae = tf.keras.metrics.MeanAbsoluteError()
# mape = tf.keras.metrics.MeanAbsolutePercentageError()
msle = tf.keras.metrics.MeanSquaredLogarithmicError()

In [68]:
metrics = {'segmentation':[tf.keras.metrics.BinaryIoU(target_class_ids = (0, 1),
                            threshold=0.5,
                            name=None,
                            dtype=None),
           tf.keras.metrics.BinaryAccuracy(
                            name='binary_accuracy', dtype=None, threshold=0.5
                        )],
           'reg_output':[mse,mae,msle]}

In [69]:
lr = 1e-5
unet_model.compile(loss={'segmentation': 'binary_crossentropy', 'reg_output': 'mse'},
              optimizer=tf.keras.optimizers.Adam(lr),metrics=metrics)